In [12]:
import pandas as pd

In [13]:
orders = pd.read_csv("olist_orders_dataset.csv")

items = pd.read_csv("olist_order_items_dataset.csv")

customers = pd.read_csv("olist_customers_dataset.csv")

sellers = pd.read_csv("olist_sellers_dataset.csv")

geo = pd.read_csv("olist_geolocation_dataset.csv")

products = pd.read_csv("olist_products_dataset.csv")

### Listed Necessary Column/Features:
#### Delivery Date (orders)
#### Estimated delivery date (orders)
####  Purchase Date (orders)
####  seller state (sellers)
####  customer state (customers)
####   product_size (products)
####   product_weight (products)
####  Distance of Delivery (derived)

In [16]:
# Get seller zip code of each order
# [order_id, seller_id]
#
orderIdSellerId = items[['order_id', 'seller_id']]

# [order_id, seller_id, seller_zip_code_prefix]
#
orderIdSellerIdWithZip = orderIdSellerId.merge(sellers[['seller_id', 'seller_zip_code_prefix']], 
                                               on = "seller_id", how = "outer")

# [order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at,
#  order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date,
#  seller_id, seller_zip_code_prefix]
#
orders = orders.merge(orderIdSellerIdWithZip, on = "order_id", how = "left")

# Get customer zip code of each order
#
# [order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at,
#  order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date,
#  seller_id, seller_zip_code_prefix, customer_zip_code_prefix]
#
orders = orders.merge(customers[['customer_id', 'customer_zip_code_prefix']],
                      on = 'customer_id', how = "left")

# Clean geo duplicates
#
geo = geo[~geo['geolocation_zip_code_prefix'].duplicated()]

# Add seller coordinates to the orders
#
# [order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at,
#  order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date,
#  seller_id, seller_zip_code_prefix, customer_zip_code_prefix, geolocation_zip_code_prefix,
#  geolocation_lat, geolocation_lng, geolocation_city, geolocation_state]
#
orders = orders.merge(geo, 
             left_on = "seller_zip_code_prefix", 
             right_on = "geolocation_zip_code_prefix", 
             how = "left")

# Add customer coordinates to the orders
#
# [order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at,
#  order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date,
#  seller_id, seller_zip_code_prefix, customer_zip_code_prefix, geolocation_zip_code_prefix_seller,
#  geolocation_lat_seller, geolocation_lng_seller, geolocation_city_seller, geolocation_state_seller,
#  geolocation_zip_code_prefix_customer, geolocation_lat_customer, geolocation_lng_customer,
#  geolocation_city_customer, geolocation_state_customer]
#
orders = orders.merge(geo, 
             left_on = "customer_zip_code_prefix", 
             right_on = "geolocation_zip_code_prefix", 
             how = "left",
             suffixes = ("_seller", "_customer"))